In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Configure Chrome options to use the proxy server
chrome_options = Options()
chrome_options.add_argument("--proxy-server=http://localhost:9919")

# Path to your ChromeDriver
driver = webdriver.Chrome(options=chrome_options)

In [3]:
def unique_elements(list1, list2):
    # Combine both lists
    combined_list = list1 + list2
    
    # Use set to find unique elements
    unique_list = list(set(combined_list))
    
    return unique_list

In [6]:
def get_page_information(link_list, result_list:list=[], visited_link_list:list=[]):
    try:
        for link in link_list:
            # Check if link already in the visited link list
            if link in visited_link_list:
                print(f'Skipping {link} as this link is already scrapped')
                continue
            else:
                visited_link_list.append(link)

            # Go to the wikipedia
            print(f'Scraping {link}')
            time.sleep(2)
            driver.get(link)

            # Get title
            # //*[@id="firstHeading"]/i
            title = driver.find_element(by = By.XPATH, value = '//*[@id="firstHeading"]').get_attribute("innerText")

            # Get content
            content = driver.find_element(by = By.XPATH, value = '//*[@id="mw-content-text"]/div[1]').text

            # Click View History
            try:
                driver.find_element(by = By.XPATH, value = '//*[@id="ca-history"]/a').click()

                # Sort by oldest update
                try:
                    driver.find_element(by = By.XPATH, value = '//*[@id="mw-content-text"]/div[3]/a[1]').click()

                    # Get created at information
                    created_at = driver.find_element(by = By.XPATH, value = '//*[@id="pagehistory"]/ul[1]/li/a').text
                except:
                    # Get created_date from the last record
                    created_at = driver.find_elements(by = By.XPATH, value = '//*[@id="pagehistory"]/ul/li/a')[-1]

                # Go back to the page
                driver.find_element(by = By.XPATH, value = '//*[@id="ca-nstab-main"]/a').click()
            except:
                created_at = ''

            # Get categories
            cat_list = driver.find_elements(by = By.XPATH, value = '//*[@id="mw-normal-catlinks"]/ul/li/a')
            categories = ''
            for cat in cat_list:
                categories += cat.text + ', '

            # Get reference links
            reference_link_list_object = driver.find_elements(by = By.XPATH, value = '//*[@id="mw-content-text"]/div[1]/div/ul/li/a')
            reference_link_list = [link.get_attribute('href') for link in reference_link_list_object]
            print(reference_link_list)

            # Store the result in dictionary
            scrape_result = {
                'title': title,
                'content': content,
                'categories': categories,
                'see_also': reference_link_list,
                'created_at': created_at,
            }

            result_list.append(scrape_result)

            # Loop
            ref_link_result = get_page_information(reference_link_list, result_list, visited_link_list)
            result_list.extend(ref_link_result)

        return result_list

    except KeyboardInterrupt:
        return result_list

In [7]:
result_list = get_page_information(['https://en.wikipedia.org/wiki/Proxy_server'])

Scraping https://en.wikipedia.org/wiki/Proxy_server
['https://en.wikipedia.org/wiki/Application_firewall', 'https://en.wikipedia.org/wiki/Captive_portal', 'https://en.wikipedia.org/wiki/Darknet', 'https://en.wikipedia.org/wiki/Distributed_Checksum_Clearinghouse', 'https://en.wikipedia.org/wiki/FreeProxy', 'https://en.wikipedia.org/wiki/Internet_privacy', 'https://en.wikipedia.org/wiki/InterPlanetary_File_System', 'https://en.wikipedia.org/wiki/Proxy_list', 'https://en.wikipedia.org/wiki/Proxy_pattern', 'https://en.wikipedia.org/wiki/SMTP_proxy', 'https://en.wikipedia.org/wiki/Virtual_private_network', 'https://en.wikipedia.org/wiki/Web_accelerator', 'https://en.wikipedia.org/wiki/Web_cache']
Scraping https://en.wikipedia.org/wiki/Application_firewall
['https://en.wikipedia.org/wiki/ModSecurity', 'https://en.wikipedia.org/wiki/Computer_security', 'https://en.wikipedia.org/wiki/Content-control_software', 'https://en.wikipedia.org/wiki/Proxy_server', 'https://en.wikipedia.org/wiki/Informa

MemoryError: 